In [ ]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from scipy.signal import find_peaks, butter, lfilter, decimate
from scipy.fft import fft, ifft
import math
import numpy as np

In [ ]:
def getPMDIRR(filename):
    df_pmdi = pd.read_csv(filename)
    df_pmdi['datetime'] = df_pmdi['TIME'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
    df_pmdi['epoch'] = df_pmdi['datetime'].apply(lambda x: datetime.datetime.timestamp(x))
    # print(df_pmdi['RESP'].unique())
    df_pmdi = df_pmdi.drop(df_pmdi[df_pmdi['RESP'] == '^^'].index)
    df_pmdi['RR'] = df_pmdi['RESP'].apply(lambda x: float(x))
    return df_pmdi[['epoch', 'RR', 'datetime']]

In [ ]:
def getDSTRR(filename):
    df_dst = pd.read_csv(filename)
    df_dst.drop(index=df_dst.index[0], axis=0, inplace=True)
    df_dst.drop_duplicates(inplace=True)
    df_dst.reset_index(drop=True, inplace=True)
    # print(df_dst.loc[~df_dst['Timestamp'].str.endswith('0000'), 'Timestamp'].apply(lambda x:))
    # df_dst['Timestamp'] = df_dst.loc[~df_dst['Timestamp'].str.contains('0000'), 'Timestamp'].apply(lambda x: x+'.000000')
    df_dst['Timestamp'] = df_dst['Timestamp'].apply(lambda x: x if x.endswith('0000') else x + '.000000')
    df_dst['Timestamp'] = df_dst['Timestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f') - pd.Timedelta('05:00:00'))
    return df_dst

In [ ]:
def selectFile():
    root = Tk()
    root.withdraw()
    root.overrideredirect(True)
    root.geometry('0x0+0+0')
    root.deiconify()
    root.lift()
    root.focus_force()
    filename = askopenfilename(filetypes=[("Comma Seperated Values File", ".csv")], parent=root, initialdir=r"\\134.117.64.31\\Main Storage")
    root.destroy()
    if not filename:
        sys.exit("No file selected")

    return filename

In [ ]:
df_PMDIRR = getPMDIRR(selectFile())
df_PMDIRR

In [ ]:
df_DSTRR = getDSTRR(selectFile())
df_test = df_DSTRR.head(32)
print(df_test)

In [ ]:
df_DSTRR['Timestamp']

In [ ]:
testRec = df_DSTRR['Timestamp'].iloc[0].to_pydatetime()
print(testRec)
print(type(testRec))

In [ ]:
df_PMDIRR_seg = df_PMDIRR.loc[df_PMDIRR['datetime'] >= df_DSTRR['Timestamp'].iloc[0].to_pydatetime()]
df_PMDIRR_seg = df_PMDIRR_seg.loc[df_PMDIRR_seg['datetime'] <= df_DSTRR['Timestamp'].iloc[-1].to_pydatetime()]
df_PMDIRR_seg.dropna(subset=['RR'], inplace=True)

In [ ]:
sns.set_theme(style='darkgrid')
g = sns.relplot(x="datetime", y="RR", kind="line", data=df_PMDIRR_seg, height=7.5)
g.figure.autofmt_xdate()

In [ ]:
# df_DSTRR_seg = df_DSTRR.loc[df_DSTRR['datetime'] >= datetime.datetime.fromisoformat('2019-01-07T12:23:40')]
print("PMDI RR first record: {}\nDST Avg Torso Depth first record: {}".format(df_PMDIRR_seg.iloc[0], df_DSTRR.iloc[0]))
print("PMDI RR last record: {}\nDST Avg Torso Depth last record: {}".format(df_PMDIRR_seg.iloc[-1], df_DSTRR.iloc[-1]))

In [ ]:
sns.set_theme(style='darkgrid')
g = sns.relplot(x='Timestamp', y='ROI Mean Depth', kind='line', data=df_DSTRR, height=7.5)
g.figure.autofmt_xdate()

In [ ]:
sns.set_theme(style='darkgrid')
g = sns.relplot(x='Timestamp', y='Total Mean Depth', kind='line', data=df_DSTRR, height=7.5)
g.figure.autofmt_xdate()

In [ ]:
df_DSTRR.head()

In [ ]:
# Use band pass filter to remove high and low frequency artifacts;
def butter_bandpass(lowCut, highCut, fs, order=2):
    nyq = 0.5 * fs
    low = lowCut/nyq
    high = highCut/nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowCut, highCut, fs, order=2):
    b, a = butter_bandpass(lowCut, highCut, fs, order=order)
    y = lfilter(b, a, data)
    return y

lowFreq = 0.35
highFreq = 1.8
fs = 20

In [ ]:
# Find RR
# Since framerate is 20 FPS, each second contains 20 samples
# Each peak needs to be a minimum of 5 samples away from the last
# Works out to a maximum BPM of 150 (Neonatal limit is around 135 BPM)
def estRR_timeDomain(df, window=150):
    estimatedRRs = []
    timestamps = []
    for window in df.rolling(window):
        windowMean = window['Mean Depth'].mean()
        window.loc[:, 'Loaded Depth'] = window['Mean Depth'].apply(lambda x: x - windowMean)
        window.loc[:, 'Filtered Signal'] = butter_bandpass_filter(window['Loaded Depth'], lowFreq, highFreq, fs)
        
        peaks, peak_props = find_peaks(window['Filtered Signal'], distance=5)
        if len(peaks) > 1:
            estRR = (((len(peaks) - 1) * 20) / (peaks[-1] - peaks[0])) * 60
            estimatedRRs.append(estRR)
            timestamps.append(window.iloc[-1]['Timestamp'])

    d = {'timestamps': timestamps, 'RR': estimatedRRs}
    df_estimatedRRs = pd.DataFrame(d)
    return df_estimatedRRs

In [ ]:
df_DSTRR_ROI = df_DSTRR[['Timestamp', 'ROI Mean Depth']].copy()
df_DSTRR_ROI.rename(columns = {'ROI Mean Depth':'Mean Depth'}, inplace = True)
df_est_td_ROI = estRR_timeDomain(df_DSTRR_ROI)
df_est_td_ROI.drop_duplicates(inplace=True)
df_est_td_ROI.reset_index(drop=True, inplace=True)


print(df_est_td_ROI.head())

In [ ]:
df_DSTRR_Total = df_DSTRR[['Timestamp', 'Total Mean Depth']].copy()
df_DSTRR_Total.rename(columns = {'Total Mean Depth':'Mean Depth'}, inplace = True)
df_est_td_Total = estRR_timeDomain(df_DSTRR_Total)
df_est_td_Total.drop_duplicates(inplace=True)
df_est_td_Total.reset_index(drop=True, inplace=True)


print(df_est_td_Total.head())

In [ ]:
def nextpow2(x):
    x = int(x)
    return 1 if x == 0 else 2**(x-1).bit_length()

In [ ]:
# Find RR using Amente's frequency domain method:
def estRR_freqDomain(df, window=1024):
    estimatedRRs = []
    timestamps = []
    for window in df.rolling(window):
        windowMean = window['Mean Depth'].mean()
        window.loc[:, 'Loaded Depth'] = window['Mean Depth'].apply(lambda x: x - windowMean)
        window.loc[:, 'Filtered Signal'] = butter_bandpass_filter(window['Loaded Depth'], lowFreq, highFreq, fs)

        L1 = len(window['Filtered Signal'])
        n1 = nextpow2(max(L1, (60*fs)))
        fft_signal = fft(window['Filtered Signal'].values, n=n1)
        f = fs * (np.array(range(int(n1/2) + 1))) / n1
        P = np.abs(fft_signal/n1)
        P1 = P[0:int(n1/2)]
        P1[1:-1] = 2 * P1[1:-1]

        largestPeak_idx = P.argmax()
        if largestPeak_idx.size != 1:
            largestPeak_idx = largestPeak_idx[0]

        estRR = f[largestPeak_idx] * 60
        estimatedRRs.append(estRR)
        timestamps.append(window.iloc[-1]['Timestamp'])

    d = {'timestamps': timestamps, 'RR': estimatedRRs}
    df_estimatedRRs = pd.DataFrame(d)
    return df_estimatedRRs

In [ ]:
df_est_fd_ROI = estRR_freqDomain(df_DSTRR_ROI)
df_est_fd_ROI.drop_duplicates(inplace=True)
df_est_fd_ROI.reset_index(drop=True, inplace=True)


print(df_est_fd_ROI.head(32))

In [ ]:
df_est_fd_Total = estRR_freqDomain(df_DSTRR_Total)
df_est_fd_Total.drop_duplicates(inplace=True)
df_est_fd_Total.reset_index(drop=True, inplace=True)


print(df_est_fd_Total.head(32))

In [ ]:
g = sns.relplot(x='timestamps', y='RR', kind='line', data=df_est_td_ROI, height=7.5)
g.figure.autofmt_xdate()

In [ ]:
g = sns.relplot(x='timestamps', y='RR', kind='line', data=df_est_td_Total, height=7.5)
g.figure.autofmt_xdate()

In [ ]:
g = sns.relplot(x='timestamps', y='RR', kind='line', data=df_est_fd_ROI, height=7.5)
g.figure.autofmt_xdate()

In [ ]:
g = sns.relplot(x='timestamps', y='RR', kind='line', data=df_est_fd_Total, height=7.5)
g.figure.autofmt_xdate()

In [ ]:
df_PMDIRR_seg.describe()

In [ ]:
df_est_td_ROI.loc[:, 'timestamps'] = df_est_td_ROI['timestamps'].apply(lambda x: x.round(freq='S'))
df_est_td_ROI.drop_duplicates(subset='timestamps', inplace=True)
df_est_td_ROI.reset_index(drop=True, inplace=True)
df_est_td_ROI.describe()

In [ ]:
df_est_td_Total.loc[:, 'timestamps'] = df_est_td_Total['timestamps'].apply(lambda x: x.round(freq='S'))
df_est_td_Total.drop_duplicates(subset='timestamps', inplace=True)
df_est_td_Total.reset_index(drop=True, inplace=True)
df_est_td_Total.describe()

In [ ]:
df_est_fd_ROI.loc[:, 'timestamps'] = df_est_fd_ROI['timestamps'].apply(lambda x: x.round(freq='S'))
df_est_fd_ROI.drop_duplicates(subset='timestamps', inplace=True)
df_est_fd_ROI.reset_index(drop=True, inplace=True)
df_est_fd_ROI.describe()

In [ ]:
df_est_fd_Total.loc[:, 'timestamps'] = df_est_fd_Total['timestamps'].apply(lambda x: x.round(freq='S'))
df_est_fd_Total.drop_duplicates(subset='timestamps', inplace=True)
df_est_fd_Total.reset_index(drop=True, inplace=True)
df_est_fd_Total.describe()

In [ ]:
df_est_td_ROI = df_est_td_ROI[df_est_td_ROI['timestamps'].isin(df_PMDIRR_seg['datetime'])]
print(df_est_td_ROI.describe())

df_est_fd_ROI = df_est_fd_ROI[df_est_fd_ROI['timestamps'].isin(df_est_td_ROI['timestamps'])]
print(df_est_fd_ROI.describe())

df_PMDIRR_seg = df_PMDIRR_seg[df_PMDIRR_seg['datetime'].isin(df_est_td_ROI['timestamps'])]
print(df_PMDIRR_seg.describe())

df_est_td_Total = df_est_td_Total[df_est_td_Total['timestamps'].isin(df_PMDIRR_seg['datetime'])]
print(df_est_td_Total.describe())

df_est_fd_Total = df_est_fd_Total[df_est_fd_Total['timestamps'].isin(df_est_td_Total['timestamps'])]
print(df_est_fd_Total.describe())

df_PMDIRR_seg = df_PMDIRR_seg[df_PMDIRR_seg['datetime'].isin(df_est_td_Total['timestamps'])]
print(df_PMDIRR_seg.describe())

In [ ]:
def MAE_rr(est_rr, pmdi_rr, sec=0):
    # Downsample records so that both sets have the same number
    # All record timestamps should be available in both sets as well
    est_rr = est_rr[est_rr['timestamps'].isin(pmdi_rr['datetime'])]
    pmdi_rr = pmdi_rr[pmdi_rr['datetime'].isin(est_rr['timestamps'])]
    # print(len(pmdi_rr))

    if sec == 0 or sec > est_rr.shape[0]:
        sec = est_rr.shape[0]

    j = 0
    MAEs = []
    while j < pmdi_rr.shape[0]:
        for i in range(j+1, pmdi_rr.shape[0]):
            if (pmdi_rr['datetime'].iloc[i] - pmdi_rr['datetime'].iloc[j]).total_seconds() >= sec:
                interval = i - j + 1
                # print(interval)
                mae = (abs(np.sum(est_rr['RR'].iloc[j:i+1].values - pmdi_rr['RR'].iloc[j:i+1].values))) / interval
                MAEs.append(mae)
                if i == pmdi_rr.shape[0] - 1:
                    j = i + 1
                else:
                    j = i
                break
            elif i == pmdi_rr.shape[0] - 1:
                j = pmdi_rr.shape[0]
                break
    return MAEs

In [ ]:
def pae_calc(mae_list, threshold=5):
    MAEs = np.array(mae_list)
    MAE_filter = MAEs <= threshold
    percentageAcceptable = (MAEs[MAE_filter].size) / MAEs.size
    return percentageAcceptable

secRange = 21

In [ ]:
print("Time domain method ROI")
PAEs_td_roi = []
for sec in range(1, secRange):
    mae = MAE_rr(df_est_td_ROI, df_PMDIRR_seg, sec=sec)
    pae = pae_calc(mae, threshold=5)
    PAEs_td_roi.append(pae)

    print("For sec={}; pae={}".format(sec, pae))

In [ ]:
print("Time domain method Total")
PAEs_td_total = []
for sec in range(1, secRange):
    mae = MAE_rr(df_est_td_Total, df_PMDIRR_seg, sec=sec)
    pae = pae_calc(mae, threshold=5)
    PAEs_td_total.append(pae)

    print("For sec={}; pae={}".format(sec, pae))

In [ ]:
print("Freq domain method ROI")
PAEs_fd_roi = []
for sec in range(1, secRange):
    mae = MAE_rr(df_est_fd_ROI, df_PMDIRR_seg, sec=sec)
    pae = pae_calc(mae, threshold=5)
    PAEs_fd_roi.append(pae)

    print("For sec={}; pae={}".format(sec, pae))

In [ ]:
print("Freq domain method Total")
PAEs_fd_total = []
for sec in range(1, secRange):
    mae = MAE_rr(df_est_fd_Total, df_PMDIRR_seg, sec=sec)
    pae = pae_calc(mae, threshold=5)
    PAEs_fd_total.append(pae)

    print("For sec={}; pae={}".format(sec, pae))

In [ ]:
df_pae_csv = pd.DataFrame({'ROI Depth TD': PAEs_td_roi}, index=range(1,21))
df_pae_csv['ROI Depth FD'] = PAEs_fd_roi
df_pae_csv['Total Depth TD'] = PAEs_td_total
df_pae_csv['Total Depth FD'] = PAEs_fd_total

In [ ]:
df_pae_csv.to_csv('df_pae.csv')

In [ ]:
df_est_td_ROI.to_csv('df_est_td_roi.csv')
df_est_fd_ROI.to_csv('df_est_fd_roi.csv')
df_est_td_Total.to_csv('df_est_td_total.csv')
df_est_fd_Total.to_csv('df_est_fd_total.csv')
df_PMDIRR_seg.to_csv('df_pmdi_seg.csv')

In [ ]:
mae = MAE_rr(df_est_fd_ROI, df_PMDIRR_seg, sec=10)

In [ ]:
def MAE_rr_seg(est_rr, pmdi_rr, sec=0):
    # Downsample records so that both sets have the same number
    # All record timestamps should be available in both sets as well
    df_acc = pd.DataFrame()
    est_rr = est_rr[est_rr['timestamps'].isin(pmdi_rr['datetime'])]
    pmdi_rr = pmdi_rr[pmdi_rr['datetime'].isin(est_rr['timestamps'])]
    print(len(pmdi_rr))

    if sec == 0 or sec > est_rr.shape[0]:
        sec = est_rr.shape[0]

    j = 0
    MAEs = []
    while j < pmdi_rr.shape[0]:
        for i in range(j+1, pmdi_rr.shape[0]):
            if (pmdi_rr['datetime'].iloc[i] - pmdi_rr['datetime'].iloc[j]).total_seconds() >= sec:
                interval = i - j + 1
                print(interval)
                mae = (abs(np.sum(est_rr['RR'].iloc[j:i+1].values - pmdi_rr['RR'].iloc[j:i+1].values))) / interval
                MAEs.append(mae)
                data = pd.DataFrame([{"i": i, "j": j, "mae": mae}])
                df_acc = df_acc.append(data)
                if i == pmdi_rr.shape[0] - 1:
                    j = i + 1
                else:
                    j = i
                break
            elif i == pmdi_rr.shape[0] - 1:
                j = pmdi_rr.shape[0]
                break
    return MAEs, df_acc

In [ ]:
# maes, df_acc = MAE_rr(df_p21_roi_fd_rr, df_p21_pmdi_rr, sec=25)

In [ ]:
# df_est_fd